<a href="https://colab.research.google.com/github/DerJFK/codewars/blob/main/math_expressions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
# https://www.codewars.com/kata/52a78825cdfc2cfc87000005

import re
import numpy as np
import pandas as pd

In [2]:
ops = {
    '*' : lambda x,y: x * y,
    '/' : lambda x,y: x / y,
    '+' : lambda x,y: x + y,
    '-' : lambda x,y: x - y
}

In [3]:
expr = (
        ("1 + 1", 2),
        ("8/16", 0.5),
        ("3 -(-1)", 4),
        ("2 + -2", 0),
        ("10- 2- -5", 13),
        ("(((10)))", 10),
        ("3 * 5", 15),
        ("-7 * -(6 / 3)", 14)
        )

In [48]:
regex = {
    'NB' : re.compile(r"\d+(\.\d*)?"),
    'PL' : re.compile(r"\("),
    'PR' : re.compile(r"\)"),
    'OP' : re.compile(r"[+\-*/]")
}
regex = pd.Series(regex)

In [5]:
class MathExpr:
  def __init__():
    pass

expr $\otimes$ expr $\otimes$ expr $\otimes$ $\ldots$

expr = (...) oder Zahl

$(-7 * -(6 / 3)) + 10 * (3 + 5)$

[,$+$]

In [60]:
e = "(-7 * -(6 / 3)) + 10 * (3 + 5)"

pL, pLR, i, j = [], [], 0, 0
lastOp, lastNB = None, None
calcOrder = []

while True:
  res = regex.apply(lambda x: x.search(e[i:]))
  idx = res.apply(lambda x: x.start() if x is not None else np.inf).idxmin()
  m = res[idx]
  if m is None:
    break

  j = i + m.start()
  i += m.end()

  if idx == 'PL':
    pL.append(j)
    continue
  if idx == 'PR':
    pLR.append((pL.pop(), i, pL[-1] if len(pL) > 0 else None))
    continue


pLR  

[(7, 14, 0), (0, 15, None), (23, 30, None)]

In [51]:
e = "(-7 * -(6 / 3)) + 10 * (3 + 5)"



<re.Match object; span=(0, 1), match='('>